# We solved it on 18.01.2023.

Let me start with the conclusion: It turns out that from the 12 IDs list, 7 are IDs used for testing (that's the best guess), and 5 are true individuals with clinical data. I believe someone did typo errors while creating the patient ID mapping file for those 5 patients, see excel file attached with the corrected User IDs.

The remaining question is: do you think the typo was done in the measurements table or in the id mapping file? My guess is it's been done in the id mapping file, see my comment on the excel sheet.

If you want more information, you can read below the summary of how I discovered it. To answer your question @Damian, the typos led to discarding ~400 FEV1 measurements and more in the other categories. It's quite an important gap given we have ~10k measurements per category in total.

Investigations' summary:
I had another look at the data for those individuals to verify that it's worth to investigate further. The conclusion is that 5 of the initial Patient IDs are worth keeping ['0HeWh64M_zc5U5l2xqzAs4', '1au5biSTt0bNWgfI0WItr5', '-TKpptiCA5cASNKU0VSmx4', '-Cujq-NEcld_Keu_W1-Nw5', '-Q0Wf614z94DSTy6nXjyw7'].  Finding their SmartCare IDs and  would increase the size of the current dataset by ~5%. Here's a summary of the measurements table with the number of recordings for each individual. --> see ## Number of recordings for USer IDs with missing SmartCare IDs

I've found potential candidates: ['38' '101' '125' '232' '169' '204']. Those IDs have clinical data but no corresponding measurements data. --> See the first table in ## Potential candidates

You can compare when first recording was made with the study start date of the potential candidates. Surprisingly there's a perfect mapping between the 4 individuals and 4 of potential candidates. The first recording is same date as the study date. 

This allows to associated each user ID with a smartcare ID, see below
ID 101: 0HeWh64M_zc5U5l2xqzAs4
ID 125: 1au5biSTt0bNWgfI0WItr5
ID 232: -TKpptiCA5cASNKU0VSmx4
ID 169: -Cujq-NEcld_Keu_W1-Nw5
ID 38: -Q0Wf614z94DSTy6nXjyw7

For each smartcare ID, I surfaced their associated User ID in the id mapping file, to compare it with the User ID from the measurements table. This revealed the typos that you can find in the excel file attached to this email.

In [9]:
# Using this because setup.py is not working
import sys; sys.path.append("../../")
import sys; sys.path.append("../data")

import measurements_data
from O2_FEV1_df import extract_measure
import plotly.express as px

df = measurements_data.load_measurements_without_smartcare_id()
df.head()


** Loading measurements data **


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/data/measurements_data.py:99: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(datadir + "mydata.csv")



* Dropping unnecessary columns from measurements data *
Columns filtered ['User ID', 'UserName', 'Recording Type', 'Date/Time recorded', 'FEV 1', 'FEV 1 %', 'Weight in Kg', 'O2 Saturation', 'Pulse (BPM)', 'Rating', 'Temp (deg C)', 'Activity - Steps', 'Activity - Points']
Dropping columns {'FEV 10', 'Sputum sample taken?', 'Calories', 'Predicted FEV'}

* Applying data sanity checks *
O2 Saturation sanity check
IDs with O2 Saturation outside 70-100 % range: 
          UserName  O2 Saturation
50403     Kings001          127.0
49087     Kings005          127.0
53923     Kings007           48.0
46706  Papworth001          127.0
46119  Papworth002          127.0
46782  Papworth010          104.0
47102  Papworth010          103.0
52484  Papworth027          127.0
53781  Papworth028          127.0
55379  Papworth028          127.0
49622   leeds01222          127.0
47510   leeds01320          127.0
Removing 12 rows with O2 Saturation outside 70-100 % range

* Looking for duplicates *
Found 464

,User ID,UserName,Recording Type,Date recorded,FEV1,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points
0,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0
1,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.0,65.0
3,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RwennLW0OTb0BG61R5R-Z4,Papworth001,ActivityRecording,2015-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2460.0,621.0


In [2]:
missing_user_ids=['o0egatM37za2ZZ2jfs3AJ5', '-Q0Wf614z94DSTy6nXjyw7'
, '31KnMK5Z4A6OKErB1C5-i6', '0HeWh64M_zc5U5l2xqzAs4'
, 'GyK_hmIGxVaT7chwlGDio7', '1au5biSTt0bNWgfI0WItr5'
, '-TKpptiCA5cASNKU0VSmx4', '-Cujq-NEcld_Keu_W1-Nw5'
, 'ci0iszS3ov6G2U-dKDiAE4', '9YevbyBS8VdwRL3RRUGY86'
, 'MB0_-sqQBxhrKyfoOxUaF4', '9MKCpOe_FO_LcU9C0oCPJ4']

fev1 = extract_measure(df, "FEV1", with_patient_id=True)
fev1.head()


FEV1 has 9949 measurements


,User ID,Date recorded,FEV1
31197,RwennLW0OTb0BG61R5R-Z4,2015-08-05,1.49
31210,RwennLW0OTb0BG61R5R-Z4,2015-08-07,1.37
31221,CpKTBsn0Hn6KpvhVNdpX56,2015-08-10,1.28
31222,VtussNQdJTdwKO4LuYroh4,2015-08-13,1.63
31233,FauQ_rxd-p7d6tmPuHhd94,2015-08-15,2.45


## Number of recordings for USer IDs with missing SmartCare IDs

In [3]:
# Counts number of measurements of FEV1 for User ID ['0HeWh64M_zc5U5l2xqzAs4', '1au5biSTt0bNWgfI0WItr5', '-TKpptiCA5cASNKU0VSmx4', '-Cujq-NEcld_Keu_W1-Nw5']
df_count=df[df["User ID"].isin(missing_user_ids)].groupby(['User ID', 'UserName']).count()
df_count=df_count[['FEV1', 'O2 Saturation', 'Weight (kg)', 'Pulse (BPM)', 'Temp (deg C)']].reset_index()
df_count['First recording']=df[df['User ID'].isin(missing_user_ids)].groupby('User ID').agg([min])['Date recorded'].values
df_count['Last recording']=df[df['User ID'].isin(missing_user_ids)].groupby('User ID').agg([max])['Date recorded'].values
# Sort by First recording
df_count=df_count.sort_values(by='First recording')
df_count[df_count['User ID'].isin(wished_ids)]

NameError: name 'wished_ids' is not defined

In [ ]:
# 303, 305, 306, 307 are the only IDs that are worht using -> with >5 measurements
# '0HeWh64M_zc5U5l2xqzAs4', '1au5biSTt0bNWgfI0WItr5', '-TKpptiCA5cASNKU0VSmx4' '-Cujq-NEcld_Keu_W1-Nw5'

# IDs with data
wished_ids = ['0HeWh64M_zc5U5l2xqzAs4', '1au5biSTt0bNWgfI0WItr5', '-TKpptiCA5cASNKU0VSmx4', '-Cujq-NEcld_Keu_W1-Nw5', '-Q0Wf614z94DSTy6nXjyw7']

# For each Patient_ID in missing_user_ids, plot FEV1
for id in wished_ids:
# for id in wished_ids:
    # Scatter plot of FEV1 measurements for this id with px
    fig = px.scatter(fev1[fev1["User ID"] == id], x="Date recorded", y="FEV1", title="FEV1 for id " + str(id))
    # Update x axis to range of FEV1 measurements 
    fig.update_xaxes(range=[fev1["Date recorded"].min(), fev1["Date recorded"].max()])
    fig.show()


In [ ]:
o2_sat = extract_measure(df, "O2 Saturation", with_patient_id=True)
o2_sat.head()


# 303, 305, 306, 307 are the only IDs that are worht using -> with >5 measurements

# For each ID in fake_ids, plot O2 saturation
for id in wished_ids:
    # Scatter plot of O2 saturation measurements for this id with px
    fig = px.scatter(df[df["User ID"] == id], x="Date recorded", y="O2 Saturation", title="O2 Saturation for id " + str(id))
    # Update x axis to range of O2 saturation measurements 
    fig.update_xaxes(range=[df["Date recorded"].min(), df["Date recorded"].max()])
    fig.show()

O2 Saturation has 14026 measurements


## Compare measurements and clinical data for those patients

In [6]:
import patient_data

df_patient = patient_data.load()


** Loading patient data **

* Dropping unnecessary columns from patient data *
Columns filtered: ['ID', 'Study Date', 'DOB', 'Age', 'Sex', 'Height', 'Weight', 'Predicted FEV1', 'FEV1 Set As']
Columns dropped: {'Unable Sputum Samples', 'CFQR Quest Comp', 'Study Email', 'GP Letter Sent', 'Study Number', 'Sputum Samples', 'Hospital', 'Date Consent Obtained', 'Age 18 Years', 'Unable Informed Consent', 'Date Last PE Stop', 'Comments', 'Less Exacerbation', 'Freezer Required', 'Remote Monitoring App User ID', 'Date Last PE Start', 'Informed Consent', 'Transplant Recipients', 'Genetic Testing', 'Telemetric Measures', 'Inconvenience Payment', 'Pulmonary Exacerbation'}

* Correcting patient data *
ID 60: Corrected height 60 from 1.63 to 163.0
ID 66: Corrected height for ID 66 from 1.62 to 162.0
Replace Age by calculate age
Replace Predicted FEV1 by the calculated version
Drop FEV1 Set As

* Applying data sanity checks *
Loaded patient data with 147 entries (147 initially)


/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning:

Unknown extension is not supported and will be removed

/Users/tristan.trebaol/Desktop/PhD/Code/old_phd/src/data/patient_data.py:137: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ID,Study Date,DOB,Age,Sex,Height,Weight,Predicted FEV1
0,36,2015-01-21,1993-11-16,21,Female,160.0,56.8,3.1950
1,23,2015-08-03,1960-07-05,55,Female,157.0,63.4,2.2265
2,24,2015-08-06,1990-11-25,25,Female,166.0,74.2,3.3320
3,138,2015-08-06,1985-10-31,30,Male,172.0,66.6,4.0360
4,80,2015-08-10,1994-08-16,21,Male,159.0,66.2,3.7380


In [7]:
df_id_map = measurements_data.load_id_map(with_correction=False)
df = measurements_data.merge_with_id_mapping(df, df_id_map)
df.head()


* Merging measurements data with ID map to retrieve SmartCare ID *
List User IDs that have no SmartCare ID
['username davetest, user id o0egatM37za2ZZ2jfs3AJ5'
 'username leeds01253, user id -Q0Wf614z94DSTy6nXjyw7'
 'username Ryan007, user id 31KnMK5Z4A6OKErB1C5-i6'
 'username Papworth016, user id 0HeWh64M_zc5U5l2xqzAs4'
 'username PapBen, user id GyK_hmIGxVaT7chwlGDio7'
 'username Papworth025, user id 1au5biSTt0bNWgfI0WItr5'
 'username BRISTOLSC003, user id -TKpptiCA5cASNKU0VSmx4'
 'username BRISTOLSC010, user id -Cujq-NEcld_Keu_W1-Nw5'
 'username scguest, user id ci0iszS3ov6G2U-dKDiAE4'
 'username PapworthSummer, user id 9YevbyBS8VdwRL3RRUGY86'
 'username Texas, user id MB0_-sqQBxhrKyfoOxUaF4'
 'username EmemTest, user id 9MKCpOe_FO_LcU9C0oCPJ4']


,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
0,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN
1,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VtussNQdJTdwKO4LuYroh4,Papworth004,ActivityRecording,2015-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.0,65.0,VtussNQdJTdwKO4LuYroh4,Papworth004,79
3,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RwennLW0OTb0BG61R5R-Z4,Papworth001,ActivityRecording,2015-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2460.0,621.0,RwennLW0OTb0BG61R5R-Z4,Papworth001,23


In [26]:
# Find IDs that are not in df but in df_patient
print(df_patient[~df_patient.ID.isin(df.ID)].ID.unique())
df_patient[~df_patient.ID.isin(df.ID)]

['38' '101' '125' '232' '169' '204']


,ID,Study Date,DOB,Age,Sex,Height,Weight,Predicted FEV1
24,38,2015-11-27,1979-09-02,36,Male,173.0,76.3,3.905
29,101,2015-12-14,1978-09-09,37,Female,166.0,61.0,3.032
49,125,2016-01-19,1984-09-19,31,Female,160.0,51.6,2.945
87,232,2016-06-02,1979-10-03,37,Female,162.0,51.9,2.874
115,169,2016-08-24,1993-11-11,23,Female,168.0,67.0,3.461
139,204,2016-10-24,1998-04-01,18,Female,150.0,57.0,2.875


In [9]:
df[~df.ID.isin(df_patient.ID)].ID.unique()

array([nan], dtype=object)

## Potential candidates 
Found by crossing clinical and measurements data

In [27]:
import pandas as pd
from measurements_data import *

id_map = measurements_data.load_id_map(with_correction=False)

# List Patient_ID for the IDs ['38' '101' '125' '232' '169' '204'] in id_map
id_map[id_map.ID.isin(['38', '101', '125', '232', '169', '204'])]


,Patient_ID,Study_ID,ID
2,TKpptiCA5cASNKU0VSmx4,BRISTOLSC003,232
9,Cujq-NEcld_Keu_W1-Nw5,BRISTOLSC010,169
69,Q0Wf614z94DSTy6nXjyw7,Leeds01253,38
111,0HeWh64M_zc5U512xqzAs4,Papworth016,101
120,1au5biSTt0bNWgfl0Wltr5,Papworth025,125
146,b4K6BRwqjMaI4QLGdM5ru7,Wessex016,204


In [13]:
# Check if '0HeWh64M_zc5U5l2xqzAs4' is as User ID in df
print((df["User ID"].eq('0HeWh64M_zc5U5l2xqzAs4')).any())
# Check if '0HeWh64M_zc5U5l2xqzAs4' is a Patient_ID in id_map
print((id_map["Patient_ID"].str.contains('0HeWh64M_zc5U5')).any())
id_map[id_map["Patient_ID"].str.contains('0HeWh64M')]


True
True


,Patient_ID,Study_ID,ID
111,0HeWh64M_zc5U512xqzAs4,Papworth016,101


In [14]:
# Check if '1au5biSTt0bNWgfl0Wltr5' is as User ID in df
print((df["User ID"].eq('1au5biSTt0bNWgfI0WItr5')).any())
# Check if '0HeWh64M_zc5U5l2xqzAs4' is a Patient_ID in id_map
print((id_map["Patient_ID"].str.contains('1au5biSTt0bNWgf')).any())
id_map[id_map["Patient_ID"].str.contains('1au5biSTt0bNWgf')]

True
True


,Patient_ID,Study_ID,ID
120,1au5biSTt0bNWgfl0Wltr5,Papworth025,125


Bingo ! 
ID in clinical data and in id map file	101	125	232	169
Patient_ID in id map file	0HeWh64M_zc5U512xqzAs4	1au5biSTt0bNWgfl0Wltr5	TKpptiCA5cASNKU0VSmx4	Cujq-NEcld_Keu_W1-Nw5
User ID in measurements data	0HeWh64M_zc5U5l2xqzAs4	1au5biSTt0bNWgfI0WItr5	-TKpptiCA5cASNKU0VSmx4	-Cujq-NEcld_Keu_W1-Nw5
Comment	1 instead of l	l instead of I	lacking a -	lacking a -

https://universityofcambridgecloud-my.sharepoint.com/:x:/r/personal/tpbt2_cam_ac_uk/Documents/2023-01-15_missing_smartcare_ids.xlsx?d=w661a7dcc9d8a40f5a18f2d1f9296fe48&csf=1&web=1&e=iMCJjk

In [15]:
# Let's check the others
['o0egatM37za2ZZ2jfs3AJ5' '-Q0Wf614z94DSTy6nXjyw7'
 '31KnMK5Z4A6OKErB1C5-i6' '0HeWh64M_zc5U5l2xqzAs4'
 'GyK_hmIGxVaT7chwlGDio7' '1au5biSTt0bNWgfI0WItr5'
 '-TKpptiCA5cASNKU0VSmx4' '-Cujq-NEcld_Keu_W1-Nw5'
 'ci0iszS3ov6G2U-dKDiAE4' '9YevbyBS8VdwRL3RRUGY86'
 'MB0_-sqQBxhrKyfoOxUaF4' '9MKCpOe_FO_LcU9C0oCPJ4']

id_map[id_map["Patient_ID"].str.contains('tM37za2ZZ')]
df[df["User ID"].eq('o0egatM37za2ZZ2jfs3AJ5')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
0,o0egatM37za2ZZ2jfs3AJ5,davetest,ActivityRecording,2015-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN


In [16]:
id_map[id_map["Patient_ID"].str.contains('Q0Wf614z94DSTy6nXjyw7')]

,Patient_ID,Study_ID,ID
69,Q0Wf614z94DSTy6nXjyw7,Leeds01253,38


In [17]:
id_map[id_map["Patient_ID"].str.contains('Z4A6OKErB1C')]
df[df["User ID"].eq('31KnMK5Z4A6OKErB1C5-i6')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
1328,31KnMK5Z4A6OKErB1C5-i6,Ryan007,ActivityRecording,2015-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336.0,85.0,NaN,NaN,NaN


In [18]:
id_map[id_map["Patient_ID"].str.contains('IGxVa')]
df[df["User ID"].eq('GyK_hmIGxVaT7chwlGDio7')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
2030,GyK_hmIGxVaT7chwlGDio7,PapBen,ActivityRecording,2016-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
id_map[id_map["Patient_ID"].str.contains('sz')]
df[df["User ID"].eq('ci0iszS3ov6G2U-dKDiAE4')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
14745,ci0iszS3ov6G2U-dKDiAE4,scguest,ActivityRecording,2017-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,38.0,NaN,NaN,NaN


In [20]:
id_map[id_map["Patient_ID"].str.contains('IGxV')]
df[df["User ID"].eq('GyK_hmIGxVaT7chwlGDio7')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
2030,GyK_hmIGxVaT7chwlGDio7,PapBen,ActivityRecording,2016-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
id_map[id_map["Patient_ID"].str.contains('ov6')]
df[df["User ID"].eq('ci0iszS3ov6G2U-dKDiAE4')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
14745,ci0iszS3ov6G2U-dKDiAE4,scguest,ActivityRecording,2017-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,38.0,NaN,NaN,NaN


In [22]:
id_map[id_map["Patient_ID"].str.contains('Vdw')]
df[df["User ID"].eq('9YevbyBS8VdwRL3RRUGY86')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
16195,9YevbyBS8VdwRL3RRUGY86,PapworthSummer,CoughRecording,2015-12-14,NaN,NaN,NaN,NaN,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
id_map[id_map["Patient_ID"].str.contains('QBx')]
df[df["User ID"].eq('MB0_-sqQBxhrKyfoOxUaF4')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
33068,MB0_-sqQBxhrKyfoOxUaF4,Texas,LungFunctionRecording,2016-03-03,1.39,2.5,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
id_map[id_map["Patient_ID"].str.contains('_FO')]
df[df["User ID"].eq('9MKCpOe_FO_LcU9C0oCPJ4')].head(1)

,User ID,UserName,Recording Type,Date recorded,FEV1,Predicted FEV,FEV1 %,Weight (kg),O2 Saturation,Pulse (BPM),Rating,Temp (deg C),Activity - Steps,Activity - Points,Patient_ID,Study_ID,ID
34164,9MKCpOe_FO_LcU9C0oCPJ4,EmemTest,LungFunctionRecording,2016-04-01,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Did damian train the predictive algorithm on the missing patients?

In [11]:
# Does the exacerbation labels data from the predictive classifier contain recording for the missing IDs?

import sys; sys.path.append("../O2-FEV1 analysis/")
import exacerbation_periods as ex

In [30]:
id_list = list(ex.get_pred_ex_labels().reset_index().ID.unique())

Exacerbated labels data from the predictive classifier has 14452 entries (1445 exacerbated, 13006 not exacerbated measurements, 1 NaN)


In [32]:
# Is 38, 101, 125, 232, 169, 204 in the list of IDs in the exacerbation labels data?
print(set(['38', '101', '125', '232', '169', '204']).issubset(set(ex.get_pred_ex_labels().reset_index().ID.unique())))

print(id_list.count('38'))
print(id_list.count('101'))
print(id_list.count('125'))
print(id_list.count('232'))
print(id_list.count('169'))
print(id_list.count('204'))

Exacerbated labels data from the predictive classifier has 14452 entries (1445 exacerbated, 13006 not exacerbated measurements, 1 NaN)
False
1
1
1
1
1
0
